In [2]:
from collections import defaultdict, deque

#function that creates our adjacency list in form of a dictionary - keys are points 'beginning pts' and entries are points
# that connect to it - gives us undirected graph sort of structure
def build_adjacency(segments):
    adj = defaultdict(set)
    for p1, p2 in segments:
        adj[p1].add(p2)
        adj[p2].add(p1)
    return adj


#uses breadth first search to actually find what components are collected. returns a 2 dimensional list, sublists are
#the endpoints included in that specific connected component.

def connected_components(adj):
    """
    adj: adjacency list
    returns: list of components, each a set of points
    """
    visited = set()
    components = []

    for start in adj.keys():
        if start in visited:
            continue

        queue = deque([start])
        comp = set()

        while queue:
            v = queue.popleft()
            if v in visited:
                continue
            visited.add(v)
            comp.add(v)
            
            for u in adj[v]:
                if u not in visited:
                    queue.append(u)

        components.append(comp)

    return components


# classifies each component by saying it is either a polygon, open chain, or lone segment (if other denoted as complex structure)
# looks at degrees of nodes, makes decisions off of that

def classify_component(component, adj):
    """
    component: set of points in the connected component
    adj: adjacency list
    returns: string classification
    """

    # degrees of each node
    degrees = [len(adj[p]) for p in component]
    edges = sum(degrees) // 2  # each edge counted twice
    nodes = len(component)

    # Closed polygon: every vertex degree=2, # edges = # nodes
    if all(d == 2 for d in degrees) and edges == nodes:
        return "closed polygon"

    # Open chain: exactly two endpoints with degree 1, others degree 2
    if degrees.count(1) == 2 and all(d in (1, 2) for d in degrees):
        if edges == nodes - 1:
            return "open chain"

    # Isolated segment: exactly one edge
    if edges == 1 and nodes == 2:
        return "isolated segment"

    # Otherwise: branching or complex network
    return "complex structure"


#for each segment, we assign it into a 'group' (what other segments belong with it in that structure) as well as a string
# classification. this essentially gives us what we want in terms of finding what segments belong to what

def group_segments(segments):
    adj = build_adjacency(segments)
    comps = connected_components(adj)

    # Map each point → component ID
    point_to_comp = {}
    for i, comp in enumerate(comps):
        for p in comp:
            point_to_comp[p] = i

    # Compute classifications
    classifications = [classify_component(comp, adj) for comp in comps]

    # Assign each segment its group and type
    results = []
    for (p1, p2) in segments:
        comp_id = point_to_comp[p1]
        results.append({
            "segment": (p1, p2),
            "group": comp_id,
            "classification": classifications[comp_id]
        })

    return results, classifications, comps




Component classifications:
  Component 0: closed polygon
  Component 1: open chain
  Component 2: open chain
  Component 3: complex structure

Segment group memberships:
{'segment': ((0, 0), (1, 0)), 'group': 0, 'classification': 'closed polygon'}
{'segment': ((1, 0), (1, 1)), 'group': 0, 'classification': 'closed polygon'}
{'segment': ((1, 1), (0, 0)), 'group': 0, 'classification': 'closed polygon'}
{'segment': ((5, 5), (6, 5)), 'group': 1, 'classification': 'open chain'}
{'segment': ((6, 5), (7, 5)), 'group': 1, 'classification': 'open chain'}
{'segment': ((10, 10), (11, 11)), 'group': 2, 'classification': 'open chain'}
{'segment': ((20, 20), (21, 20)), 'group': 3, 'classification': 'complex structure'}
{'segment': ((21, 20), (22, 20)), 'group': 3, 'classification': 'complex structure'}
{'segment': ((21, 20), (21, 21)), 'group': 3, 'classification': 'complex structure'}


In [4]:
#example

segments = [
    # Closed triangle
    ((0,0), (1,0)),
    ((1,0), (1,1)),
    ((1,1), (0,0)),
    
    # Open chain
    ((5,5), (6,5)),
    ((6,5), (7,5)),

    # Isolated segment
    ((10,10), (11,11)),

    # Complex branched structure
    ((20,20), (21,20)),
    ((21,20), (22,20)),
    ((21,20), (21,21)),
]

results, classifications, comps = group_segments(segments)

print("Component classifications:")
for i, c in enumerate(classifications):
    print(f"  Component {i}: {c}")

print("\nSegment group memberships:")
for r in results:
    print(r)

Component classifications:
  Component 0: closed polygon
  Component 1: open chain
  Component 2: open chain
  Component 3: complex structure

Segment group memberships:
{'segment': ((0, 0), (1, 0)), 'group': 0, 'classification': 'closed polygon'}
{'segment': ((1, 0), (1, 1)), 'group': 0, 'classification': 'closed polygon'}
{'segment': ((1, 1), (0, 0)), 'group': 0, 'classification': 'closed polygon'}
{'segment': ((5, 5), (6, 5)), 'group': 1, 'classification': 'open chain'}
{'segment': ((6, 5), (7, 5)), 'group': 1, 'classification': 'open chain'}
{'segment': ((10, 10), (11, 11)), 'group': 2, 'classification': 'open chain'}
{'segment': ((20, 20), (21, 20)), 'group': 3, 'classification': 'complex structure'}
{'segment': ((21, 20), (22, 20)), 'group': 3, 'classification': 'complex structure'}
{'segment': ((21, 20), (21, 21)), 'group': 3, 'classification': 'complex structure'}
